In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
import dotenv

sys.path.append('../')

import src.qa_gen as qag

dotenv.load_dotenv(os.environ['HOME'] + '/profile.env', override=True)


True

In [2]:
# Other models to try here:
# https://console.groq.com/docs/models
GROQ_MODEL = "llama-3.1-70b-versatile"
# GROQ_MODEL = "llama-3.1-70b-specdec"

qa_generator = qag.QAGenerator(
                    api_keys_var="GROQ_API_KEYS",
                    prompt_ver="spa-v0",
                    groq_model=GROQ_MODEL,
                    pause_secs=1.1, # No more than 60 reqs per minute.
                    cache_enabled=False
                )

Returning secret from environment variable `GROQ_API_KEYS`=`gs...a9`
6 Groq api keys loaded.
QAGenerator.init: prompt_tmpl=
'''
Por favor genera una pregunta de opción múltiple (con una única respuesta correcta y 3 respuestas incorrectas) a partir del fragmento de texto que te doy más abajo.

Es decir, el formato que esperamos es:

```
Pregunta: <pregunta aquí, no más de 10 o 15 palabras y termina en signo de interrogación>
Respuesta correcta: <respuesta correcta aquí>
Respuesta incorrecta 1: <respuesta incorrecta 1 aquí>
Respuesta incorrecta 2: <respuesta incorrecta 2 aquí>
Respuesta incorrecta 3: <respuesta incorrecta 3 aquí>
```

El texto a partir del cuál debes generar la pregunta es:
```
{chunk}
```
Pregunta: '''


In [3]:
chunk_eng = """
The corporation shall be dissolved:
For reasons stipulated by law.
By decision of the Superior Council, taken by two-thirds of its ordinary councilors in office, in two sessions of the Superior Council that must take place with an interval of no less than thirty calendar days, when in their judgment the Institution is unable to fulfill its purposes.
"""

chunk_spa="""
La Universidad tendrá un revisor fiscal, designado por el Consejo Superior para períodos de dos años. La función de revisor fiscal deberá ser ejercida por una sociedad profesional del ramo.
"""

generated_qa = qa_generator.gen_question(chunk_spa, verbose=True)


==========QA GENERATION PROMPT=====

Por favor genera una pregunta de opción múltiple (con una única respuesta correcta y 3 respuestas incorrectas) a partir del fragmento de texto que te doy más abajo.

Es decir, el formato que esperamos es:

```
Pregunta: <pregunta aquí, no más de 10 o 15 palabras y termina en signo de interrogación>
Respuesta correcta: <respuesta correcta aquí>
Respuesta incorrecta 1: <respuesta incorrecta 1 aquí>
Respuesta incorrecta 2: <respuesta incorrecta 2 aquí>
Respuesta incorrecta 3: <respuesta incorrecta 3 aquí>
```

El texto a partir del cuál debes generar la pregunta es:
```

La Universidad tendrá un revisor fiscal, designado por el Consejo Superior para períodos de dos años. La función de revisor fiscal deberá ser ejercida por una sociedad profesional del ramo.

```
Pregunta: 

=== GENERATED QUESTION AND ANSWERS ====
Pregunta: ¿Quién designa al revisor fiscal de la Universidad?
Respuesta correcta: El Consejo Superior
Respuesta incorrecta 1: La Asamblea Uni

In [14]:
in_rec = qag.parse_generated_question(generated_qa=generated_qa, match_strs=qa_generator.match_strs)
in_rec

{'question': '¿Quién designa al revisor fiscal de la Universidad?',
 'correct_answer': ' El Consejo Superior.',
 'incorrect_answers': ['El Rector de la Universidad.',
  'La Asamblea Nacional.',
  'El Ministro de Educación.']}

In [12]:
import src.utils as ut
import json

def generate_qas_file(qa_generator: qag.QAGenerator,
                      input_path: Path,
                      src_file_key: str,
                      start_idx: int,
                      out_fpath: Path):
    lines = input_path.read_text().split("\n")
    print(len(lines), "lines")

    with out_fpath.open("wt") as f_out:
        for i, (s, e, src_chunk) in enumerate(ut.chunk_generator(lines, start_idx=start_idx, min_chunk_len=300)):
            print(f"{i:4d} - start:{s:4d} end:{e:4d} len:{len(src_chunk)} "
                f"{src_chunk[:45]!r}...{src_chunk[-45:]!r}")

            generated_qa = qa_generator.gen_question(src_chunk, verbose=(i<10))

            qa_record = qag.parse_generated_question(generated_qa=generated_qa,
                                                     match_strs=qa_generator.match_strs)
            example_id = f"{src_file_key}-s{s}-e{e}"
            out_rec = qag.enrich_generated_qa(qa_record, src_chunk, example_id)
            print(json.dumps(out_rec), file=f_out)


In [10]:

qa_generator = qag.QAGenerator(
    api_keys_var="GROQ_API_KEYS",
    groq_model=GROQ_MODEL,
    pause_secs=1.0,
    prompt_ver="spa-v0",
    cache_enabled=True
)


Returning secret from environment variable `GROQ_API_KEYS`=`gs...a9`
6 Groq api keys loaded.
QAGenerator.init: prompt_tmpl=
'''
Por favor genera una pregunta de opción múltiple (con una única respuesta correcta y 3 respuestas incorrectas) a partir del fragmento de texto que te doy más abajo.
La pregunta NO DEBE ser sobre números o títulos de artículos o capítulos.

Es decir, el formato que esperamos es:

```
Pregunta: <pregunta aquí, no más de 10 o 15 palabras y termina en signo de interrogación>
Respuesta correcta: <respuesta correcta aquí>
Respuesta incorrecta 1: <respuesta incorrecta 1 aquí>
Respuesta incorrecta 2: <respuesta incorrecta 2 aquí>
Respuesta incorrecta 3: <respuesta incorrecta 3 aquí>
```

El texto a partir del cuál debes generar la pregunta es:
```
{chunk}
```
Pregunta: '''


In [ ]:
# file_preffix = "Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.preprocessed2"
file_preffix = "reglamento-maestria-web-2024.preprocessed2"

generate_qas_file(
    qa_generator,
    input_path = Path(f"../data/{file_preffix}.txt"),
    src_file_key = "estatutos",
    start_idx = 0,
    out_fpath = Path(f"../data/{file_preffix}.qa.jsonl")
)


417 lines
   0 - start:   0 end:  10 len:967 '## ESTATUTOS UNIVERSIDAD DE LOS ANDES\n\nBogotá'...'nata de perfección que hay en el ser humano”.'
Found in cache!
   1 - start:   2 end:  10 len:928 'Bogotá, 2020\n\nLa versión actualizada de los e'...'nata de perfección que hay en el ser humano”.'
==========QA GENERATION PROMPT=====

Por favor genera una pregunta de opción múltiple (con una única respuesta correcta y 3 respuestas incorrectas) a partir del fragmento de texto que te doy más abajo.
La pregunta NO DEBE ser sobre números o títulos de artículos o capítulos.

Es decir, el formato que esperamos es:

```
Pregunta: <pregunta aquí, no más de 10 o 15 palabras y termina en signo de interrogación>
Respuesta correcta: <respuesta correcta aquí>
Respuesta incorrecta 1: <respuesta incorrecta 1 aquí>
Respuesta incorrecta 2: <respuesta incorrecta 2 aquí>
Respuesta incorrecta 3: <respuesta incorrecta 3 aquí>
```

El texto a partir del cuál debes generar la pregunta es:
```
Bogotá, 2020

La ve

In [ ]:
generate_qas_file(
    qa_generator,
    input_path = Path("../data/reglamento-maestria-web-2024.preprocessed2.txt"),
    src_file_key = "maestria",
    start_idx = 130,
    out_fpath = Path("../data/reglamento-maestria-web-2024.preprocessed2.qa.jsonl")
)


687 lines
   0 - start: 130 end: 134 len:372 '\n## DEFINITIONS\n\nUniversidad de los Andes is '...' well as their commitment to the environment.'
==========QA GENERATION PROMPT=====

Please generate one multiple choice question, a correct answer for it and three (3) incorrect answers, based on the following text:
```

## DEFINITIONS

Universidad de los Andes is an autonomous, independent, and innovative institution that fosters pluralism, tolerance, and respect for ideas; it seeks academic excellence and provides its students with a critical and ethical education to strengthen their awareness of their social and civic responsibilities, as well as their commitment to the environment.
```
The question should be short, i.e. between about 8 and 15 words long and end on question mark, and should be contained in a SINGLE line
DO NOT preface the question with any introductory remark, except "QUESTION:"
The question should not be about chapter/section/article numbers or titles.
You should retu

In [ ]:
keys = [ key for key in qa_generator.cache.keys() if 'When a second call is necessary,' in key]
print(len(keys))
for k in keys:
    del qa_generator.cache[k]

0


In [ ]:
generated_qa = """Here's the generated question based on the provided text.

QUESTION: What type of relationships does the institution develop?
CORRECT ANSWER: academic, scientific, artistic
INCORRECT ANSWER 1: Only financial relationships
 INCORRECT ANSWER 2: No relationships whatsoever
INCORRECT ANSWER 3: Sports relationships
"""

qag.parse_generated_question(generated_qa)

{'question': "Here's the generated question based on the provided text. What type of relationships does the institution develop?",
 'correct_answer': 'academic, scientific, artistic',
 'incorrect_answers': ['Only financial relationships',
  'No relationships whatsoever',
  'Sports relationships']}